# Simple Search Engine
Elasticsearch와 mysql을 결합한 검색엔진을 만들어보겠습니다.
____

In [1]:
import pandas as pd
df = pd.read_csv("movie_doc.csv",sep="\t")

In [2]:
df.head()

,id,title,story,gender,link,image
0,0,인비저블 게스트,"의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 ...",N,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20170828_179/1...
1,1,"나, 다니엘 블레이크",평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20161117_246/1...
2,2,국가부도의 날,"1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, 곧 엄청난 경제...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20181105_232/1...
3,3,당갈,전직 레슬링 선수였던 ‘마하비르 싱 포갓(아미르 칸)’은 아버지의 반대로 금메달의 ...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20180329_2/152...
4,4,스파이더맨: 파 프롬 홈,"‘엔드게임’ 이후 변화된 세상, 스파이더맨 ‘피터 파커’는 학교 친구들과 유럽 여행...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20190527_181/1...


## 2. MySQL에 데이터 적재

In [3]:
# import mysql.connector


# HOST = "localhost"
# USER = "root"
# PASSWORD = "Boost111@"
# mydb = mysql.connector.connect(
#   host=HOST,
#   user=USER,
#   password=PASSWORD,
#   database="movie_db"
# )

# mycursor = mydb.cursor()


In [4]:
# sql = "SELECT * FROM movie"
# mycursor = mydb.cursor(dictionary=True)
# mycursor.execute(sql)


# myresult = mycursor.fetchall()
# for x in myresult:
#     print(x)
#     break

## 3. ElasticSearch에 데이터 인덱싱

In [5]:
from elasticsearch import Elasticsearch, helpers


In [6]:
import pprint  
INDEX_NAME = "movie_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ]
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "story" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


In [7]:


try:
    es.transport.close()
except:
    pass
es = Elasticsearch()



In [8]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-8-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-8-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_index'}

In [10]:
# movies = [
#     {
#         "_index": INDEX_NAME,
#         "_id" : doc['id'],
#         "_source": {
#             "title": doc['title'],
#             "story": doc['story'],
#         }
#     }
#     for doc in myresult
# ]

# movies[:10]

In [11]:
try:
    response = helpers.bulk(es, movies)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)


ERROR: name 'movies' is not defined


In [13]:
# doc = es.get(index=INDEX_NAME, id=1)
# pprint.pprint(doc)

## 4. ElasticSearch에서 검색후, 검색결과를 MySQL에서 재 검색

In [14]:
query="은퇴한 킬러"
res = es.search(index=INDEX_NAME, q=query)
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))
    print("Title: {}".format(hit['_source']['title']))
    print("줄거리: {}".format(hit['_source']['story']))


In [15]:
movie_id = res['hits']['hits'][0]["_id"]

IndexError: list index out of range

In [16]:
sql = "SELECT * FROM movie WHERE id = {}".format(movie_id)
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

NameError: name 'movie_id' is not defined

In [ ]:
#쿼리---> 엘라스틱서치 --> MySQL